In [ ]:
!pip install sentencepiece
!pip install transformers

ERROR: Operation cancelled by user


In [ ]:
# !pip install sentencepiece
# !pip install transformers

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import pandas as pd
import nltk
import datetime
import numpy as np
# from transformers import 
from transformers import M2M100Config, M2M100ForConditionalGeneration, M2M100Tokenizer, AdamW


cpu


In [ ]:
# MOUNT DRIVE TO SAVE OUTPUTS

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# https://colab.research.google.com/drive/1-JIJlao4dI-Ilww_NnTc0rxtp-ymgDgM?usp=sharing
# https://huggingface.co/transformers/custom_datasets.html#additional-resources


In [ ]:
!nvidia-smi
# https://huggingface.co/transformers/model_doc/m2m_100.html

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# get data
! git clone https://github.com/zachary-m/gc.git
! git clone https://github.com/yoraish/bibrew.git

# Can change nikud below to include/not include it.
# df = pd.read_pickle("/content/gc/train_test_val_nikud.pkl")
df = pd.read_pickle("/content/bibrew/train_test_val_nikud.pkl")
(df.src + "\t" + df.target).to_csv("bibheb-modernheb.txt", index=False, header=None)

fatal: destination path 'gc' already exists and is not an empty directory.
fatal: destination path 'bibrew' already exists and is not an empty directory.


In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained('facebook/m2m100_418M')
tokenizer = M2M100Tokenizer.from_pretrained('facebook/m2m100_418M', src_lang="he", tgt_lang="he")

In [ ]:
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
model = model.train()

In [ ]:
train = df.loc[df["data_type"] == "train", ["src", "target"]]
val = df.loc[df["data_type"] == "val", ["src", "target"]]
test = df.loc[df["data_type"] == "test", ["src", "target"]]

In [ ]:
df

,src,target,data_type
0,וַיִּֽהְיוּ שָֽׁם שְׂדֵֽי מוֹאָ֖ב וַיָּבֹ֥אוּ ...,שָׁם וְנִשְׁאֲרוּ מוֹאָב לְאֶרֶץ בָּאוּ הֵם שֶ...,test
1,הַתֶּֽבֶן בִּהְי֥וֹת כַּאֲשֶׁ֖ר בְּיוֹמ֔וֹ דְּ...,הַקַּשׁ אֶת שֶׁקִּבַּלְתֶּם בַּזְּמַן יוֹם בְּ...,test
2,וָמַֽשׁ וְגֶ֥תֶר וְח֖וּל ע֥וּץ אֲרָ֑ם וּבְנֵ֖י,וּמַשׁ וְגֶתֶר וְחוּל עוּץ אֲרָם וּבְנֵי,test
3,סָבִֽיב הַמִּזְבֵּ֖חַ עַל קַרְנ֥וֹת וְנָתַ֛ן ה...,מִסָּבִיב הַמִּזְבֵּחַ פִּנּוֹת עַל וְיָשִׂים ...,test
4,תְעוֹנֵֽנוּ וְלֹ֥א תְנַחֲשׁ֖וּ לֹ֥א עַל הַדָּ֑...,עֵינַיִם תֹּאחֲזוּ וְאַל תְּכַשְּׁפוּ אַל הַדָ...,test
...,...,...,...
9217,יָשָֽׁן תֹּאכְל֖וּ תְּב֣וּאָתָ֔הּ עַד בּוֹא֙ ה...,הַיְּשָׁנָה מֵהַתְּבוּאָה תֹּאכְלוּ שָׁנָה אוֹ...,train
9218,בְּמוֹעֲדָֽם אֹתָ֖ם אֲשֶׁר תִּקְרְא֥וּ קֹ֑דֶשׁ...,בִּזְמַנָּם אוֹתָם שֶׁתִּקְבְּעוּ קוֹדֶשׁ מְאו...,train
9219,הָאֶחָֽד לָאַ֖יִל עֶשְׂרֹנִ֔ים שְׁנֵי֙ לַפָּ֔ר...,אַיִל לְכָל לִיטֶר כְּ6 5 הָאֵיפָה עֲשִׂירִיּו...,train
9220,הַיִּשְׂרְאֵלִֽי וְאִ֖ישׁ הַיִּשְׂרְאֵלִ֔ית בֶ...,בַּמַּחֲנֶה רָבוּ יִשְׂרְאֵלִי וְאִישׁ הַיִּשְ...,train


In [ ]:
# HOW DO I GET THIS TOWORK???
# from datasets import load_metric
# metric = load_metric("bleu")

In [ ]:
# Get the BLEU score before doing any training.
model.eval()
bleus_epoch = []

for i, row in tqdm(list(val.iterrows())):
  optimizer.zero_grad(False)
  src, target = row["src"], row["target"]
  encoded_hi = tokenizer(src, return_tensors="pt").to(device)
  generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
  encode_target = tokenizer(target, return_tensors="pt").to(device)
  encode_target = encode_target['input_ids'].squeeze().cpu().detach().numpy()
  predicted = generated_tokens.squeeze().cpu().detach().numpy()
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([encode_target], predicted, weights = [1])
  bleus_epoch.append(BLEUscore) # record in the epoch


mean_bleu = np.mean(bleus_epoch)
print("Epoch: NO TRAIN  Mean Val BLEU {}".format(mean_bleu))

In [ ]:
from google.colab import files

RUN_NAME = "training"
LOG_FILE_NAME = 'log_hugface_model_yorai.csv'
DRIVE_PATH = "/content/gdrive/MyDrive/"
LOG_FILE_FULL = DRIVE_PATH + LOG_FILE_NAME

model_name = "ftmt_nikud"

tokenizer.src_lang = "he"
losses = []
bleus = []
best_bleu = -1
best_parameters = None
for epoch in list(range(10)):
  model_path = '/content/gdrive/MyDrive/school/current_school/6.806/NLPProject/src/models/{}.pth'.format(model_name + "_epoch_"+str(epoch))

  model.train() # am I supposed to do this each time?
  epoch_loss = []
  bleus_epoch = []
  # TRAIN
  print("-"*8 + " Epoch {} ".format(epoch) + "-"*8)
  for i, row in tqdm(list(train.sample(frac=1).iterrows())):  # maybe increase the batch size?
    src, target = row["src"], row["target"]
    model_inputs = tokenizer(src, return_tensors="pt").to(device)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target, return_tensors="pt").input_ids
    labels = labels.to(device)
    # print(labels)
    optimizer.zero_grad()
    loss = model(**model_inputs, labels=labels)# forward pass
    loss.loss.backward()
    optimizer.step()
    epoch_loss.append(loss.loss.item()) # add to loss records
    # if i % 1000 == 0:
      # print("Epoch: {}  Loss: {}".format(epoch, loss.loss))

  # RECORD INFO!
  dt = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
  df_vls = [epoch, "train", "loss", dt, RUN_NAME] + epoch_loss
  pd.DataFrame(df_vls).T.to_csv(LOG_FILE_FULL , mode='a', index=False, header=False)

  print("Epoch: {}  Mean Training Loss: {}".format(epoch, np.mean(epoch_loss)))
  losses.append(epoch_loss) # keep track of all losses
  model.eval()
  
  # VAL BLEU!
  # THIS SHOULD BE MUCH FASTER, HOW TONOT USE NLTK??
  for i, row in tqdm(list(val.iterrows())):
    optimizer.zero_grad(False)
    src, target = row["src"], row["target"]
    encoded_hi = tokenizer(src, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
    encode_target = tokenizer(target, return_tensors="pt").to(device)
    encode_target = encode_target['input_ids'].squeeze().cpu().detach().numpy()
    predicted = generated_tokens.squeeze().cpu().detach().numpy()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([encode_target], predicted, weights = [1])
    bleus_epoch.append(BLEUscore) # record in the epoch



  # RECORD INFO!
  dt = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
  df_vls = [epoch, "val", "BLUE", dt, RUN_NAME] + bleus_epoch
  pd.DataFrame(df_vls).T.to_csv(LOG_FILE_FULL, mode='a', index=False, header=False)
  torch.save(model.state_dict(), model_path)

  bleus.append(bleus_epoch) # record all
  mean_bleu = np.mean(bleus_epoch)

  # record model parameters of best model
  if  mean_bleu > best_bleu:
    best_bleu = mean_bleu
    best_parameters = model.parameters()
    # model.save
  
  print("Epoch: {}  Mean Val BLEU {}".format(epoch, mean_bleu))



-------- Epoch 0 --------


Epoch: 0  Loss: 1.364159107208252
Epoch: 0  Loss: 1.1714987754821777
Epoch: 0  Loss: 2.098445415496826
Epoch: 0  Loss: 1.2903000116348267
Epoch: 0  Loss: 1.0695443153381348
Epoch: 0  Loss: 0.8630951642990112
Epoch: 0  Loss: 1.1802972555160522
Epoch: 0  Loss: 1.0444930791854858

Epoch: 0  Mean Training Loss: 1.0559585036411792



Epoch: 0  Mean Val BLEU 0.7300957842684809
-------- Epoch 1 --------


Epoch: 1  Loss: 0.6886698603630066
Epoch: 1  Loss: 0.9258601665496826
Epoch: 1  Loss: 0.9366827607154846
Epoch: 1  Loss: 0.7117842435836792
Epoch: 1  Loss: 0.9996904134750366
Epoch: 1  Loss: 0.4915826916694641
Epoch: 1  Loss: 1.738523244857788
Epoch: 1  Loss: 0.5488230586051941

Epoch: 1  Mean Training Loss: 0.7167797056641886



Epoch: 1  Mean Val BLEU 0.7490858535522135
-------- Epoch 2 --------


Epoch: 2  Loss: 0.5446508526802063
Epoch: 2  Loss: 0.7807838916778564
Epoch: 2  Loss: 0.43781277537345886
Epoch: 2  Loss: 0.43445107340812683
Epoch: 2  Loss: 0.6305311918258667
Epoch: 2  Loss: 0.6894408464431763
Epoch: 2  Loss: 1.5903453826904297
Epoch: 2  Loss: 0.5651273131370544

Epoch: 2  Mean Training Loss: 0.577062620987483



Epoch: 2  Mean Val BLEU 0.7634960540955712
-------- Epoch 3 --------


Epoch: 3  Loss: 0.45644474029541016
Epoch: 3  Loss: 0.3892825245857239
Epoch: 3  Loss: 1.275073528289795
Epoch: 3  Loss: 0.5142726302146912
Epoch: 3  Loss: 0.5536128878593445
Epoch: 3  Loss: 0.7933863997459412
Epoch: 3  Loss: 0.6826207637786865
Epoch: 3  Loss: 0.376758873462677

Epoch: 3  Mean Training Loss: 0.48206402954136124



Epoch: 3  Mean Val BLEU 0.773466040894721
-------- Epoch 4 --------


Epoch: 4  Loss: 0.3600058853626251
Epoch: 4  Loss: 0.37893185019493103
Epoch: 4  Loss: 0.40095236897468567
Epoch: 4  Loss: 0.2329261302947998
Epoch: 4  Loss: 1.2970470190048218
Epoch: 4  Loss: 0.6285766959190369
Epoch: 4  Loss: 0.3451840281486511
Epoch: 4  Loss: 0.6019630432128906

Epoch: 4  Mean Training Loss: 0.40478017085787904


In [ ]:
from google.colab import files
model_name = "ftmt_nikud"
model_path = '/content/gdrive/MyDrive/school/current_school/6.806/NLPProject/src/models/{}.pth'.format(model_name)
torch.save(model.state_dict(), model_path)
files.download(model_path)

In [ ]:
  df_vls = [epoch, "train", "loss", dt, RUN_NAME] + epoch_loss
  pd.DataFrame(df_vls).T.to_csv('/content/gdrive/MyDrive/log_hugface_model.csv', mode='a', index=False, header=False)

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
pd.DataFrame(df_vls)

,0
0,epoch 0
1,train
2,loss
3,"05/12/2021, 18:25:12"
4,TEST
5,1.53829
6,1.38626
7,0.0727819
8,0.180002
9,1.11663


# get model outputs

In [ ]:
model.eval()
encoded_hi = tokenizer(src, return_tensors="pt").to(device)
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
print("orignal")
print(src)
print("TARGET")
print(target)
print("RESULT")
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

orignal
ובהר בבקעה יצאים ותהמת עינת מים נחלי ארץ טובה אל ארץ מביאך אלהיך יהוה כי
TARGET
ובהר בבקעה שנובעים ותהומות מעינות מים נחלי ארץ טובה לארץ אותך מביא אלהיך יהוה כי
RESULT


['ובהר בבקעה יוצאים ותהומות מעינות מים נחלי ארץ טובה לארץ אותך מביא אלהיך יהוה כי']

# Trying to get it to train in batches

In [ ]:
tokenizer.src_lang = "he"
losses = []
bleus = []

best_bleu = -1
best_parameters = None
for epoch in list(range(10)):
  model.train() # am I supposed to do this each time?
  epoch_loss = []
  bleus_epoch = []
  # TRAIN
  train_ = train.sample(frac=1)
  index_batches = np.array_split(train.index, 230) # batch size of 32
  for indexes  in tqdm(index_batches):  # maybe increase the batch size?
    train_batch = train_.iloc[indexes]
    src, target = train_batch[["src"]].values, train_batch[["target"]].values  # get these columns and the numpy arrays for them
    model_inputs = tokenizer(src, return_tensors="pt").to(device)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target, return_tensors="pt").input_ids
    labels = labels.to(device)
    # print(labels)
    optimizer.zero_grad()
    loss = model(**model_inputs, labels=labels)# forward pass
    loss.loss.backward()
    optimizer.step()
    # break
    epoch_loss.append(loss.loss.item()) # add to loss records
    if i % 1000 == 0:
      print("Epoch: {}  Loss: {}".format(epoch, loss.loss))
    break

  print("Epoch: {}  Mean Training Loss: {}".format(epoch, np.mean(epoch_loss)))
  losses.append(epoch_loss) # keep track of all losses
  model.eval()
  
  # VAL BLEU!
  
  for i, row in tqdm(list(val.iterrows())):
    optimizer.zero_grad(False)
    src, target = row["src"], row["target"]
    encoded_hi = tokenizer(src, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
    encode_target = tokenizer(target, return_tensors="pt").to(device)
    encode_target = encode_target['input_ids'].squeeze().cpu().detach().numpy()
    predicted = generated_tokens.squeeze().cpu().detach().numpy()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([encode_target], predicted, weights = [1])
    bleus_epoch.append(BLEUscore) # record in the epoch
    break

  bleus.append(bleus_epoch) # record all
  mean_bleu = np.mean(bleus_epoch)

  # record model parameters of best model
  if  mean_bleu > best_bleu:
    best_bleu = mean_bleu
    best_parameters = model.parameters()
  
  print("Epoch: {}  Mean Val BLEU {}".format(epoch, mean_bleu))

